# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import psutil

train_data = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 10000)
max_session = metadata['max_session']

with tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave= False) as rambar:

  for i in tqdm(range(0, num_frags)):

    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))
    rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)
    rambar.refresh()




RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/720 [00:00<?, ?it/s]

In [3]:
train_data = pd.concat(train_data)

In [4]:
candidates = train_data
candidates.shape

(39240555, 240)

In [5]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [6]:
candidates

,user,item,num_clicks,num_carts,num_orders,item_degree,item_pr,recent_num_clicks,recent_num_carts,recent_num_orders,item_recent_degree,item_recent_pr,recent_day21_type0,recent_day21_type1,recent_day21_type2,recent_day20_type0,recent_day20_type1,recent_day20_type2,recent_day19_type0,recent_day19_type1,recent_day19_type2,recent_day18_type0,recent_day18_type1,recent_day18_type2,recent_day17_type0,recent_day17_type1,recent_day17_type2,recent_day16_type0,recent_day16_type1,recent_day16_type2,recent_day15_type0,recent_day15_type1,recent_day15_type2,recent_day14_type0,recent_day14_type1,recent_day14_type2,recent_day13_type0,recent_day13_type1,recent_day13_type2,recent_day12_type0,recent_day12_type1,recent_day12_type2,recent_day11_type0,recent_day11_type1,recent_day11_type2,recent_day10_type0,recent_day10_type1,recent_day10_type2,recent_day9_type0,recent_day9_type1,recent_day9_type2,recent_day8_type0,recent_day8_type1,recent_day8_type2,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2,item_glob_last_action,item_glob_first_action,item_glob_time_decay_sum,item_glob_time_decay_sum_click,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_order,item_glob_item_glob_durability,qou_num_cosub_mean,qou_coclick_sub_coef_mean,qou_cocart_sub_coef_mean,qou_coorder_sub_coef_mean,qou_coclick_time_decay_mean,qou_cocart_time_decay_mean,qou_coorder_time_decay_mean,qou_num_appearance_mean,qou_num_in_k_most_recent_items_mean,qou_num_happend_later_mean,qou_num_happend_before_mean,qou_happend_later_ratio_mean,qou_last_interact_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_3_1_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_sub_coef_0.5_10_0.5_mean,qou_lincom_time_decay_0.5_10_0.5_mean,qou_lincom_sub_coef_10_0.5_0.5_mean,qou_lincom_time_decay_10_0.5_0.5_mean,qou_lincom_sub_coef_0.5_0.5_10_mean,qou_lincom_time_decay_0.5_0.5_10_mean,qou_num_cosub_sqrt_num_cousers,qou_coclick_sub_coef_sqrt_num_cousers,qou_cocart_sub_coef_sqrt_num_cousers,qou_coorder_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_sqrt_num_cousers,qou_cocart_time_decay_sqrt_num_cousers,qou_coorder_time_decay_sqrt_num_cousers,qou_num_appearance_sqrt_num_cousers,qou_num_in_k_most_recent_items_sqrt_num_cousers,qou_num_happend_later_sqrt_num_cousers,qou_num_happend_before_sqrt_num_cousers,qou_happend_later_ratio_sqrt_num_cousers,qou_last_interact_sqrt_num_cousers,num_sub,consistency,num_actions,user_degree,user_pr,user_recent_degree,user_recent_pr,cointeractions_coef_clicks_clicks,cointeractions_coef_clicks_carts,cointeractions_coef_clicks_orders,cointeractions_coef_carts_clicks,cointeractions_coef_carts_carts,cointeractions_coef_carts_orders,cointeractions_coef_orders_clicks,cointeractions_coef_orders_carts,cointeractions_coef_orders_orders,cointeractions_time_decay_clicks_clicks,cointeractions_time_decay_clicks_carts,cointeractions_time_decay_clicks_orders,cointeractions_time_decay_carts_clicks,cointeractions_time_decay_carts_carts,cointeractions_time_decay_carts_orders,cointeractions_time_decay_orders_clicks,cointeractions_time_decay_orders_carts,cointeractions_time_decay_orders_orders,fitness,inter_clicks,inter_carts,inter_orders,inter_num_sub,inter_time_decay,inter_lts,inter_fts,inter_durability,inter_num_interacts,is_level1,user_clicks_cnt,user_carts_cnt,user_orders_cnt,user_total_actions_cnt,user_clicks_ratio,user_carts_ratio,user_orders_ratio,user_ss_ts_max

# Train params

In [7]:
from meow_utils import *
from catboost.utils import eval_metric
learning_rate = [0.5, 0.25, 0.25]
early_stopping_rounds = [20, 50,50]
PARAMS = {
    'iterations': 1000,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': 'RecallAt:top=20;border=0.5',
    'use_best_model': True,
    'thread_count': -1,
}
N_SPLITS = 5
NUM_SAMPLES = 5
NUM_CLICK_SAMPLES = 2
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '13'

# Train for each type

In [8]:
from meow_utils import get_len_group

In [9]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [10]:
labels = []
users = candidates.user.values
items = candidates.item.values
from tqdm.notebook import tqdm
for i in tqdm(range(3)):
  cnt = 0
  tar_small = tar[tar['type'] == i].sort_values(['user', 'item']).set_index(['user', 'item'])
  
  tar_small_index = set(tar_small.index.tolist())
  label = []

  for i in tqdm(range(candidates.shape[0]), leave = False):
    if (users[i], items[i]) in tar_small_index:
      label.append(1)
      cnt += 1
    else: 
      label.append(0)
  labels.append(label)
  print(f'Found {cnt} positives', end = ' ')
del tar, tar_small_index
  



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/39240555 [00:00<?, ?it/s]

Found 301362 positives 

  0%|          | 0/39240555 [00:00<?, ?it/s]

Found 76615 positives 

  0%|          | 0/39240555 [00:00<?, ?it/s]

Found 57516 positives 

In [11]:
labels = np.array(labels)

In [12]:
from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
import shutil
from meow_utils import plot_importance
from catboost import CatBoostRanker, Pool
import seaborn as sns
sns.set()
%matplotlib inline
def train(t, id_type, draw_importance = False):
  global candidates, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  fig, axs = plt.subplots(1, N_SPLITS, figsize = (300, 30))
  skf = GroupKFold(n_splits=N_SPLITS)
  
  
  X_all = candidates.iloc[:, 2 :]
  y_all = labels[id_type]
  print('Building pool...')
  data_all = Pool(X_all, y_all, thread_count = os.cpu_count())
  print('Pool builed!!!!!!')
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):
      neg_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 0))
      pos_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 1))
      #downsample many times
      for sample in range(NUM_SAMPLES if id_type != 0 else NUM_CLICK_SAMPLES):
        print(f'FOLD: {fold} Sample: {sample}')
        n_index = np.random.permutation(neg_index)[:len(pos_index) * 30]
        # n_index = neg_index
        index = np.union1d(n_index, pos_index)
        
        data_train = data_all.slice(train_idx).slice(index)
        data_train.set_group_id(candidates['user'].values.astype(int)[train_idx][index])
        data_valid = data_all.slice(valid_idx)
        data_valid.set_group_id(candidates['user'].values.astype(int)[valid_idx])

        params = {'thread_count' : os.cpu_count(),
                  'early_stopping_rounds': early_stopping_rounds[id_type],
                  'learning_rate': learning_rate[id_type],
        **PARAMS}
        model = CatBoostRanker(**params)
        
        model.fit(data_train, eval_set = data_valid)
        
        try:
          os.mkdir(MODEL_PATH + name + VERSION)
        except:
          pass
        if fold < 5 and sample == 0 and draw_importance:
          plot_importance(X_all.columns.tolist(), model.get_feature_importance(data = data_valid, type = 'LossFunctionChange'), ax = axs[fold])
          axs[fold].bar_label(axs[fold].containers[0])
        model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}_sample{sample}')
  


In [13]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(t, id_type, draw_importance = False)

remove old version
Building pool...
Pool builed!!!!!!
FOLD: 0 Sample: 0
0:	learn: 0.9983680	test: 0.8987509	best: 0.8987509 (0)	total: 1.84s	remaining: 30m 39s
1:	learn: 0.9988790	test: 0.9111133	best: 0.9111133 (1)	total: 3.21s	remaining: 26m 40s
2:	learn: 0.9993254	test: 0.9164469	best: 0.9164469 (2)	total: 4.54s	remaining: 25m 8s
3:	learn: 0.9994172	test: 0.9196788	best: 0.9196788 (3)	total: 5.85s	remaining: 24m 17s
4:	learn: 0.9995015	test: 0.9206206	best: 0.9206206 (4)	total: 7.13s	remaining: 23m 39s
5:	learn: 0.9995263	test: 0.9223059	best: 0.9223059 (5)	total: 8.47s	remaining: 23m 23s
6:	learn: 0.9995288	test: 0.9225240	best: 0.9225240 (6)	total: 9.79s	remaining: 23m 8s
7:	learn: 0.9995560	test: 0.9233568	best: 0.9233568 (7)	total: 11.4s	remaining: 23m 34s
8:	learn: 0.9996081	test: 0.9239021	best: 0.9239021 (8)	total: 12.7s	remaining: 23m 19s
9:	learn: 0.9996230	test: 0.9242887	best: 0.9242887 (9)	total: 14.1s	remaining: 23m 13s
10:	learn: 0.9996379	test: 0.9246654	best: 0.92466

Training has stopped (degenerate solution on iteration 73, probably too small l2-regularization, try to increase it)



bestTest = 0.9972747047
bestIteration = 28

Shrink model to first 29 iterations.
FOLD: 1 Sample: 0
0:	learn: 0.9999980	test: 0.9944752	best: 0.9944752 (0)	total: 539ms	remaining: 8m 58s
1:	learn: 0.9999981	test: 0.9957020	best: 0.9957020 (1)	total: 1.08s	remaining: 8m 58s
2:	learn: 0.9999981	test: 0.9959844	best: 0.9959844 (2)	total: 1.6s	remaining: 8m 50s
3:	learn: 0.9999984	test: 0.9963707	best: 0.9963707 (3)	total: 2.06s	remaining: 8m 31s
4:	learn: 0.9999986	test: 0.9966193	best: 0.9966193 (4)	total: 2.51s	remaining: 8m 18s
5:	learn: 0.9999987	test: 0.9967266	best: 0.9967266 (5)	total: 2.97s	remaining: 8m 12s
6:	learn: 0.9999987	test: 0.9967696	best: 0.9967696 (6)	total: 3.58s	remaining: 8m 28s
7:	learn: 0.9999987	test: 0.9967759	best: 0.9967759 (7)	total: 4.07s	remaining: 8m 24s
8:	learn: 0.9999987	test: 0.9967989	best: 0.9967989 (8)	total: 4.58s	remaining: 8m 23s
9:	learn: 0.9999987	test: 0.9968682	best: 0.9968682 (9)	total: 5.11s	remaining: 8m 25s
10:	learn: 0.9999986	test: 0.99

Training has stopped (degenerate solution on iteration 72, probably too small l2-regularization, try to increase it)



bestTest = 0.9972349156
bestIteration = 47

Shrink model to first 48 iterations.
